### Obvious things to tweak


Number of layers
Nodes per layer
Dense layer at the end or not

In [16]:
import time
dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

3-conv-32-nodes0-dense-1540613255
3-conv-64-nodes0-dense-1540613255
3-conv-128-nodes0-dense-1540613255
3-conv-32-nodes1-dense-1540613255
3-conv-64-nodes1-dense-1540613255
3-conv-128-nodes1-dense-1540613255
3-conv-32-nodes2-dense-1540613255
3-conv-64-nodes2-dense-1540613255
3-conv-128-nodes2-dense-1540613255


In [18]:
import tensorflow as tf
import keras
from keras import backend as K

# NOTE: To shrink the VRAM fraction add this to ConfigProto
# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
# K.tf.ConfigProto(gpu_options=gpu_options)

cfg = K.tf.ConfigProto(device_count = {'CPU' : 1, 'GPU' : 1})
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.utils import normalize
from keras.callbacks import TensorBoard

import pickle
import time

X = pickle.load(open("data/pickled/X.pickle", "rb"))
y = pickle.load(open("data/pickled/y.pickle", "rb"))

X = keras.utils.normalize(X, axis=1)

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            K.clear_session()
            NAME = "{}-conv-{}-nodes{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir='logs_r2/{}'.format(NAME))
            
            model = Sequential()

            # the 128 can be anything
            # 3,3 is the window of the convolution (3 pixels X 3 pixels)
            # X.shape[1:] is 128, 128, 1 based ont he fact that I transformed the images to 128X128 pixes
            model.add(Conv2D(layer_size, (3,3), input_shape = X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))
            
            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3,3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))
            
            model.add(Flatten())
            for l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation("relu"))

            # Output Layer
            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            model.compile(loss='binary_crossentropy',
                         optimizer='adam',
                         metrics=['accuracy'])

            model.fit(X, y, batch_size=32, epochs=10, validation_split=0.1, callbacks=[tensorboard])

Train on 22451 samples, validate on 2495 samples
Epoch 1/10
22451/22451 [==============================] - 17s 759us/step - loss: 0.6488 - acc: 0.6194 - val_loss: 0.6195 - val_acc: 0.6425
Epoch 2/10
22451/22451 [==============================] - 17s 745us/step - loss: 0.5812 - acc: 0.7025 - val_loss: 0.5884 - val_acc: 0.6910
Epoch 3/10
22451/22451 [==============================] - 17s 751us/step - loss: 0.5173 - acc: 0.7517 - val_loss: 0.4794 - val_acc: 0.7792
Epoch 4/10
22451/22451 [==============================] - 17s 751us/step - loss: 0.4756 - acc: 0.7782 - val_loss: 0.4830 - val_acc: 0.7671
Epoch 5/10
22451/22451 [==============================] - 17s 759us/step - loss: 0.4517 - acc: 0.7936 - val_loss: 0.4388 - val_acc: 0.7980
Epoch 6/10
22451/22451 [==============================] - 17s 749us/step - loss: 0.4280 - acc: 0.8061 - val_loss: 0.4574 - val_acc: 0.7820
Epoch 7/10
22451/22451 [==============================] - 17s 752us/step - loss: 0.4084 - acc: 0.8127 - val_loss: 0.4

22451/22451 [==============================] - 54s 2ms/step - loss: 0.1349 - acc: 0.9480 - val_loss: 0.9484 - val_acc: 0.7315
Epoch 9/10
22451/22451 [==============================] - 54s 2ms/step - loss: 0.0763 - acc: 0.9723 - val_loss: 1.1931 - val_acc: 0.7230
Epoch 10/10
22451/22451 [==============================] - 54s 2ms/step - loss: 0.0421 - acc: 0.9860 - val_loss: 1.3845 - val_acc: 0.7311
Train on 22451 samples, validate on 2495 samples
Epoch 1/10
22451/22451 [==============================] - 18s 794us/step - loss: 0.6932 - acc: 0.5003 - val_loss: 0.6931 - val_acc: 0.5134
Epoch 2/10
22451/22451 [==============================] - 17s 774us/step - loss: 0.6932 - acc: 0.4972 - val_loss: 0.6931 - val_acc: 0.5138
Epoch 3/10
22451/22451 [==============================] - 17s 768us/step - loss: 0.6932 - acc: 0.4987 - val_loss: 0.6932 - val_acc: 0.4866
Epoch 4/10
22451/22451 [==============================] - 17s 772us/step - loss: 0.6933 - acc: 0.4950 - val_loss: 0.6933 - val_acc: 0